In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

* The first step is to import packages that will be required

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score

# Read Dataset

In [ ]:
df = pd.read_csv("/kaggle/input/credit-card-customers/BankChurners.csv")
df.head()

* Last two columns name are too long, so we will change it them

In [ ]:
df_new = df.rename(columns={'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1': 'NBC_Mon_1',
                           'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2':'NBC_Mon_2'})
df_new.head()

#  Info About Data

* There is no null value in any column

In [ ]:
df_new.isnull().sum()

* Numbers of non-unique data found, we saw CLIENTNUM column is nunique that's mean every row it has a different value, it's private customer number, we don't need to because machine learning algorithm it can memorize.

In [ ]:
df_new.nunique()

* Show data distrubition in graph

In [ ]:
sbn.histplot(x=df_new["Attrition_Flag"],data = df_new,hue=df_new["Attrition_Flag"])

# Preprocessing

In [ ]:
#CLIENTNUM is non unique has values so we can't take for train dataset
df_X = df_new.drop(["CLIENTNUM","Attrition_Flag"],axis = 1)

* Next step we will do label encoding(LabelEncoder) for all columns

In [ ]:
X = df_X.apply(LabelEncoder().fit_transform).values 
Y = df["Attrition_Flag"].values.ravel()
print("X Shape: ",X.shape)
print("Y Shape: ",Y.shape)

# Train and Test Split

* Split data train and test

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

* Next step is show a new train and test shape  

In [ ]:
print("X Train Shape: ",x_train.shape)
print("Y Train Shape: ",y_train.shape)
print("X Test Shape: ",x_test.shape)
print("Y Test Shape: ",y_test.shape)

# Scaling of data

* With MinMaxScaler is to scale between 1-0 

In [ ]:
sc = MinMaxScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

# Training 

* Train the data with K-NN 

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5 ,metric='minkowski')
knn.fit(X_train,y_train)

# Evaluate The Model

* Next step, firstly we need to predict X_test with K-NN and this result will be our predictions. Here we will call it y_pred.

In [ ]:
y_pred = knn.predict(X_test)

* Confusion Matrix
* Diagonal is our true positive and true negative values

In [ ]:
cm = confusion_matrix(y_pred,y_test)
print("Confusion Matrix Result: ")
print(cm)

* Next step, show in graph the confusion matrix 

In [ ]:
sbn.heatmap(cm,annot=True,fmt='d',linecolor = 'white',linewidths=5)

* Accuracy Score as a percent

In [ ]:
acc = accuracy_score(y_test,y_pred)
print("Accuracy Score : ",acc*100)

* K-fold Cross Validation 

In [ ]:
success = cross_val_score(estimator = knn ,X=X_train,y=y_train,cv=4)
# Cross validation result print
print(success.mean())
print(success.std())
print(success.max())